# 1.1-agifford-VisualizeSingleDataFile
This notebook tests out visualizing samples of data aligned to physical actiity starts and ends, and templates a set of functions to reproduce a visualization for any dataframe and any time period.

In [ ]:
import pandas as pd
import plotly.express as px

parq_file = "../../data/interim/raw/fileID1_subjID3_dataID0.parquet"
df = pd.read_parquet(parq_file, engine="fastparquet")

In [ ]:
df.head()

In [ ]:
df_ = df[(df.time > 400) & (df.time <= 500)]
activity_starts_df = df_[df_.label != df_.label.shift(1)]
activity_ends_df = df_[df_.label != df_.label.shift(-1)]

activity_starts_df = activity_starts_df.dropna(subset="label")
activity_ends_df = activity_ends_df.dropna(subset="label")

In [ ]:
blue = ["blue" for _ in df_]
green = ["green" for _ in df_]
red = ["red" for _ in df_]
fig = px.line(
    df_, 
    x="time", 
    y="accel_x", 
    color_discrete_sequence=blue, 
    title="Acceleration Traces Sample"
)
fig.add_traces(list(
    px.line(
        df_, 
        x="time", 
        y="accel_y", 
        color_discrete_sequence=green
    ).select_traces()
))
fig.add_traces(list(
    px.line(
        df_, 
        x="time", 
        y="accel_z", 
        color_discrete_sequence=red
    ).select_traces()
))

for x in activity_starts_df.index:
    fig.add_vline(
        x=activity_starts_df.loc[x, "time"], 
        annotation_text=activity_starts_df.loc[x, "label"]
    )
for x in activity_ends_df.index:
    fig.add_vline(
        x=activity_ends_df.loc[x, "time"], line_dash="dash",
    )
    
fig.update_layout(xaxis_range=[398,502])
for d, name in zip(fig.data, ["accel_x", "accel_y", "accel_z"]):
    d["showlegend"] = True
    d["name"] = name

fig.update_yaxes(title_text="Acceleration")
fig.show()

In [ ]:
df_mlt_accl = df_.melt(id_vars=["time"], value_vars=["accel_x", "accel_y", "accel_z"])
df_mlt_accl["measure"] = "acceleration"
df_mlt_gryo = df_.melt(id_vars=["time"], value_vars=["gyro_x", "gyro_y", "gyro_z"])
df_mlt_gryo["measure"] = "gryoscope"
df_melt = pd.concat([df_mlt_accl, df_mlt_gryo], ignore_index=True)

fig = px.line(
    df_melt, 
    x="time", 
    y="value", 
    color="variable", 
    facet_col="measure", 
    title="Acceleration Traces Sample", 
    facet_col_spacing=0.05
)

for x in activity_starts_df.index:
    fig.add_vline(
        x=activity_starts_df.loc[x, "time"], 
        annotation_text=activity_starts_df.loc[x, "label"]
    )
for x in activity_ends_df.index:
    fig.add_vline(
        x=activity_ends_df.loc[x, "time"], line_dash="dash",
    )

fig.update_yaxes(matches=None)
fig.for_each_yaxis(lambda yaxis: yaxis.update(showticklabels=True))
fig.for_each_xaxis(lambda xaxis: xaxis.update(range=[398, 502]))
fig.show()

In [ ]:
def _make_single_annot_frame(df, shift):
    annot_df = df[df.label != df.label.shift(shift)]
    annot_df = annot_df.dropna(subset="label")
    return annot_df

def make_annot_dataframe(df, t_start=None, t_end=None):
    t_start = df.time.min() or t_start
    t_end = df.time.max() or t_end
    
    df = df[(df.time > t_start) & (df.time <= t_end)].copy()
    
    (act_starts_df, act_ends_df) = (
        _make_single_annot_frame(df, shift) for shift in [1, -1]
    )
    return act_starts_df, act_ends_df

def _melt_single_frame(df, meas):
    x, y, z = (meas + i for i in ["_x", "_y", "_z"])
    df_melted = df.melt(id_vars=["time"], value_vars=[x, y, z])
    
    measure = "acceleration" if "accel" in meas else "gyroscope"
    df_melted["measure"] = measure
    return df_melted

def _melt_dataframe(df):
    (df_mlt_accl, df_mlt_gryo) = (
        _melt_single_frame(df, meas) for meas in ["accel", "gyro"]
    )
    df_melted = pd.concat([df_mlt_accl, df_mlt_gryo], ignore_index=True)
    return df_melted

def make_annotated_trace(df, t_start, t_end):
    
    activity_starts_df, activity_ends_df = make_annot_dataframe(df, t_start, t_end)
    df_melted = _melt_dataframe(df)
    fig = px.line(
        df_melted, 
        x="time", 
        y="value", 
        color="variable", 
        facet_col="measure", 
        title="Data Traces Sample", 
        facet_col_spacing=0.05, 
        width=1000, 
        height=400
    )

    for x in activity_starts_df.index:
        fig.add_vline(
            x=activity_starts_df.loc[x, "time"], 
            annotation_text=activity_starts_df.loc[x, "label"]
        )
    for x in activity_ends_df.index:
        fig.add_vline(
            x=activity_ends_df.loc[x, "time"], line_dash="dash",
        )

    fig.update_yaxes(matches=None)
    fig.for_each_yaxis(lambda yaxis: yaxis.update(showticklabels=True))
    x_min = t_start - 0.005*t_start
    x_max = t_end + (0.005* 0.99)*t_end
    fig.for_each_xaxis(lambda xaxis: xaxis.update(range=[x_min, x_max]))
    return fig

In [ ]:
fig = make_annotated_trace(df, t_start=400, t_end=600)
fig.show()